### export homologue SNP regions in BED files for Bam read counts 

Rename the chromosome to UCSC




example: chr1    13649   13649


1. add  one to offset the zero-based from UCSC (zero based left)
2. Take the window size of 2bp upstream and 2bp downstream


In [1]:
import pandas as pd
import os
import numpy as np

baseChainDir='/cellar/users/btsui/Data/ucsc_chains_human_homo_snps/'

fnameS=pd.Series(os.listdir(baseChainDir))
#yeast, fruit flies, worms, zebrafish, and mice
chainNameToSpecieDict={ 
                      #"":'Saccharomyces_cerevisiae', #R64-1-1.
                      #"":'Drosophila_melanogaster', #BDGP6
                      #"":'Caenorhabditis_elegans',#WBcel235/ CE11
                      "hg38ToDanRer10.over.chain.gz.human_homo.bed":"Danio_rerio",#GRCz10,  DanRer10
                       'hg38ToMm10.over.chain.gz.human_homo.bed':'Mus_musculus'
                      }

#m_test=fnameS.str.contains('Mm10')
#m_mapped=fnameS.str.contains('human_homo.bed$')
#inFnameS=fnameS[m_mapped]

In [2]:
#Saccharomyces_cerevisiae.R64-1-1.dna_rm.toplevel.fa
#!ls $baseChainDir

In [3]:
colOrder=['Chr','Start','End','Rs']

#for fname in inFnameS:
#    break
fname='hg38ToDanRer10.over.chain.gz.human_homo.bed'

### do for one species for now
specie=chainNameToSpecieDict[fname]

liftOverDf_all=pd.read_csv(baseChainDir+fname,sep='\t',header=None)
liftOverDf_all.columns=colOrder
liftOverDf_all['Chr']=liftOverDf_all['Chr'].astype(str)

In [4]:
m=~liftOverDf_all['Chr'].str.contains('_')
liftOverDf=liftOverDf_all[m]


In [5]:
exportCols=['Chr','Start_offset','End_offset']


In [6]:
#liftOverDf

In [7]:
liftOverDf['Chr']=liftOverDf['Chr'].str.replace('M','MT').str.replace('chr','')

offset=2
liftOverDf['Start_one_based']=liftOverDf['Start']+1
liftOverDf['Start_offset']=liftOverDf['Start_one_based']-offset
liftOverDf['End_offset']=liftOverDf['Start_one_based']+offset
liftOverDf['Start_offset'][liftOverDf['Start_offset']<=0]=0

#!head /cellar/users/btsui/Data/dbsnp/snp_beds/Homo_sapiens.bed

baseGenomesDir='/cellar/users/btsui/Data/BOWTIE_GENOME_SNP_INDEX/'
snpBed='/cellar/users/btsui/Data/dbsnp/snp_beds/'+specie+'.bed'

#!ls /cellar/users/btsui/Data/dbsnp/snp_beds/

liftOverDf[exportCols].sort_values(exportCols).to_csv(snpBed,sep='\t',header=None,index=False)

/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [8]:
#!head $snpBed

### generate refernce

### extract the SNP mapping window  window 

In [9]:
snpBed='extracting_region.bed'
###do for dbSNP
offset=1000
liftOverDf['Start_one_based']=liftOverDf['Start']+1
liftOverDf['Start_offset']=liftOverDf['Start_one_based']-offset
liftOverDf['Start_offset'][liftOverDf['Start_offset']<=0]=0
liftOverDf['End_offset']=liftOverDf['Start_one_based']+offset
liftOverDf.sort_values(exportCols,ascending=True)[exportCols].to_csv(snpBed,sep='\t',header=None,index=False)

/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-

In [10]:
#liftOverDf[liftOverDf.Chr=='MT']

In [11]:
#2537074256/2725537669.0

In [12]:
#liftOverDf.sort_values(['Chr','Start_offset'],ascending=False)[exportCols]['Chr'].value_counts()

In [13]:
#index the genome
myCleanFaDir='/cellar/users/btsui/Data/ensembl/clean/{}.fa'.format(specie)
os.system(' samtools faidx '+ myCleanFaDir)

0

In [14]:
#!cat /cellar/users/btsui/Data/ensembl/clean/Mus_musculus.fa.fai

In [15]:
myFai=pd.read_csv(myCleanFaDir+'.fai',sep='\t',header=None)
myFai[0]=myFai[0].astype(str)
myFai[[0,1]].sort_values(0).to_csv('genome',sep='\t',header=None,index=None)

In [16]:
!bedtools complement -i extracting_region.bed -g genome > complement.txt

***** WARNING: 8:54189719-54192493 exceeds the length of chromosome (8)
***** WARNING: MT:0-17563 exceeds the length of chromosome (MT)


### masking all the regions complementing to the tasking windows with Ns

In [17]:
complementDf=pd.read_csv('complement.txt',sep='\t',header=None)

In [18]:
complementDf.columns=['Chr','Start','End']

In [19]:
spaceMasked=(complementDf.End-complementDf.Start)

In [55]:
outMaskedFastaDir='/cellar/users/btsui/Data/ensembl/snp_masked/'

outMaskedFaDir=outMaskedFastaDir+specie+'.fa'
targeted_fast_gz=outMaskedFaDir+'.gz'

In [56]:
os.system('maskFastaFromBed -fi {myCleanFaDir} -bed complement.txt -fo {maskedFasta}'.format(
myCleanFaDir=myCleanFaDir,
maskedFasta=outMaskedFaDir
))

0

In [57]:
!pigz --force $outMaskedFaDir

### concat viral signatures from Methplan and 16s genes from mothur

https://www.mothur.org/wiki/RDP_reference_files

## bacteria and microbes reference mothur: 
1. trainset16_022016.rdp.tax : id to sequence tax mapping
2. trainset16_022016.rdp.fasta.gz

## viral 
1. ftp://ftp.ncbi.nih.gov/refseq/release/viral/viral.1.1.genomic.fna.gz
2. ftp://ftp.ncbi.nih.gov/refseq/release/viral/viral.2.1.genomic.fna.gz

microbe consist of viral and microbial genome from NCBI and mothur respectively

In [58]:
withMicrobeDir=targeted_fast_gz.replace('.fa.gz','.microbe.fa.gz')
cmd_cat='cat microbe.fa.gz {targeted_fast_gz} > {withMicrobeDir}'.format(targeted_fast_gz=targeted_fast_gz,
                                                                          withMicrobeDir=withMicrobeDir)


In [59]:
!$cmd_cat

In [60]:
#os.system(cmd_cat)

In [61]:
#!rm /cellar/users/btsui/Data/ensembl/snp_masked/Danio_rerio.microbe.fa.gz 

In [63]:
!gunzip --keep --force $withMicrobeDir

In [64]:
os.system(' samtools faidx '+ withMicrobeDir.replace('.fa.gz','.fa'))

0

In [65]:
#!tail /cellar/users/btsui/Data/ensembl/snp_masked/Mus_musculus.microbe.fa.fai

In [66]:
bowtieFaDir=withMicrobeDir

In [67]:
bowtie2IndexPrefix='/cellar/users/btsui/Data/BOWTIE_GENOME_SNP_INDEX/{specie}/bowtie2'.format(specie=specie)

In [68]:
bowtie2IndexDir=bowtie2IndexPrefix.replace('/bowtie2','')

In [69]:
!mkdir $bowtie2IndexDir

mkdir: cannot create directory ‘/cellar/users/btsui/Data/BOWTIE_GENOME_SNP_INDEX/Danio_rerio’: File exists


In [70]:
cmd= 'bowtie2-build --threads 64 '+bowtieFaDir+' '+bowtie2IndexPrefix
!$cmd

Settings:
  Output files: "/cellar/users/btsui/Data/BOWTIE_GENOME_SNP_INDEX/Danio_rerio/bowtie2.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 256
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /cellar/users/btsui/Data/ensembl/snp_masked/Danio_rerio.microbe.fa.gz
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:14
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:17
bmax according to bmaxDivN setting: 1228139
Using parameters --bmax 921105 --dcv 1024
  Doing ahead-of-time mem

  Entering block accumulator loop for bucket 59:
  Entering block accumulator loop for bucket 53:
  Entering block accumulator loop for bucket 54:
  Entering block accumulator loop for bucket 57:
  Entering block accumulator loop for bucket 13:
  Entering block accumulator loop for bucket 30:
  Entering block accumulator loop for bucket 39:
  Entering block accumulator loop for bucket 34:
  Entering block accumulator loop for bucket 16:
  Entering block accumulator loop for bucket 40:
  Entering block accumulator loop for bucket 41:
  Entering block accumulator loop for bucket 32:
  Entering block accumulator loop for bucket 64:
  Entering block accumulator loop for bucket 31:
  Entering block accumulator loop for bucket 33:
  Entering block accumulator loop for bucket 35:
  Entering block accumulator loop for bucket 38:
  Entering block accumulator loop for bucket 56:
  Entering block accumulator loop for bucket 62:
  Entering block accumulator loop for bucket 63:
  Entering block acc

  bucket 23: 70%
  bucket 51: 60%
  bucket 14: 70%
  bucket 8: 70%
  bucket 12: 70%
  bucket 21: 70%
  bucket 44: 60%
  bucket 13: 70%
  bucket 16: 70%
  bucket 42: 60%
  bucket 36: 60%
  bucket 34: 70%
  bucket 37: 70%
  bucket 10: 70%
  bucket 4: 70%
  bucket 1: 80%
  bucket 56: 60%
  bucket 25: 70%
  bucket 6: 70%
  bucket 24: 70%
  bucket 30: 70%
  bucket 38: 70%
  bucket 9: 70%
  bucket 11: 70%
  bucket 46: 70%
  bucket 48: 60%
  bucket 61: 60%
  bucket 57: 60%
  bucket 55: 60%
  bucket 45: 70%
  bucket 43: 70%
  bucket 32: 70%
  bucket 63: 60%
  bucket 28: 70%
  bucket 26: 70%
  bucket 22: 70%
  bucket 20: 70%
  bucket 62: 60%
  bucket 60: 60%
  bucket 19: 70%
  bucket 15: 70%
  bucket 47: 70%
  bucket 29: 80%
  bucket 31: 70%
  bucket 35: 70%
  bucket 3: 80%
  bucket 27: 80%
  bucket 59: 60%
  bucket 50: 70%
  bucket 2: 80%
  bucket 18: 70%
  bucket 41: 70%
  bucket 33: 70%
  bucket 53: 70%
  bucket 7: 80%
  bucket 5: 80%
  bucket 39: 80%
  bucket 17: 80%
  bucket 8: 80%
  bucke

  bucket 65: 10%
  bucket 40: 100%
  Sorting block of length 372840 for bucket 40
  (Using difference cover)
  bucket 64: 100%
  Sorting block of length 716833 for bucket 64
  (Using difference cover)
  bucket 44: 100%
  Sorting block of length 553233 for bucket 44
  (Using difference cover)
  Sorting block time: 00:00:01
Returning block of 873162 for bucket 21
  bucket 54: 100%
  Sorting block of length 776410 for bucket 54
  (Using difference cover)
  Sorting block time: 00:00:01
Returning block of 849983 for bucket 3
  bucket 50: 100%
  Sorting block of length 174229 for bucket 50
  (Using difference cover)
  bucket 36: 100%
  Sorting block of length 531991 for bucket 36
  (Using difference cover)
  Sorting block time: 00:00:00
Returning block of 790377 for bucket 14
  bucket 42: 100%
  Sorting block of length 540904 for bucket 42
  (Using difference cover)
  bucket 49: 100%
  Sorting block of length 760783 for bucket 49
  (Using difference cover)
  bucket 59: 90%
Getting block 69 o

  Sorting block time: 00:00:01
Returning block of 602460 for bucket 47
  Sorting block time: 00:00:01
Returning block of 553234 for bucket 44
  Sorting block time: 00:00:01
Returning block of 410667 for bucket 61
  Sorting block time: 00:00:01
Returning block of 644546 for bucket 31
  bucket 65: 30%
  Sorting block time: 00:00:01
Returning block of 720699 for bucket 52
  Sorting block time: 00:00:00
Returning block of 546146 for bucket 60
  Sorting block time: 00:00:01
Returning block of 540905 for bucket 42
  Sorting block time: 00:00:01
Returning block of 807266 for bucket 28
  Sorting block time: 00:00:01
Returning block of 776411 for bucket 54
  Sorting block time: 00:00:01
Returning block of 579132 for bucket 55
  Sorting block time: 00:00:01
Returning block of 760748 for bucket 33
  Sorting block time: 00:00:01
Returning block of 716834 for bucket 64
  Sorting block time: 00:00:00
Returning block of 782637 for bucket 48
  Sorting block time: 00:00:01
Returning block of 760784 for

...

Getting block 416 of 463
  Reserving size (921105) for bucket 416
  Calculating Z arrays for bucket 416
  Entering block accumulator loop for bucket 416:
  bucket 372: 90%
  bucket 381: 70%
Getting block 417 of 463
  Reserving size (921105) for bucket 417
  Calculating Z arrays for bucket 417
  Entering block accumulator loop for bucket 417:
  bucket 365: 100%
  Sorting block of length 671382 for bucket 365
  (Using difference cover)
  bucket 373: 90%
  bucket 379: 80%
  bucket 374: 90%
  bucket 376: 80%
  bucket 366: 100%
  Sorting block of length 304204 for bucket 366
  (Using difference cover)
  Sorting block time: 00:00:00
Returning block of 802766 for bucket 352
  bucket 385: 70%
  bucket 407: 10%
  bucket 380: 80%
  bucket 399: 30%
  Sorting block time: 00:00:00
Returning block of 198292 for bucket 358
  bucket 369: 100%
  Sorting block of length 447395 for bucket 369
  (Using difference cover)
  bucket 386: 70%
  bucket 377: 100%
  Sorting block of length 528991 for bucket 377
 

  Sorting block time: 00:00:01
Returning block of 877877 for bucket 373
  bucket 419: 20%
  bucket 417: 20%
  bucket 389: 90%
  bucket 399: 50%
Getting block 439 of 463
  Reserving size (921105) for bucket 439
  Calculating Z arrays for bucket 439
  Entering block accumulator loop for bucket 439:
  bucket 407: 30%
  bucket 431: 10%
  bucket 430: 10%
Getting block 440 of 463
  Reserving size (921105) for bucket 440
  Calculating Z arrays for bucket 440
  Entering block accumulator loop for bucket 440:
  bucket 416: 20%
  bucket 394: 60%
  bucket 410: 30%
  bucket 432: 10%
  bucket 379: 100%
  Sorting block of length 736277 for bucket 379
  (Using difference cover)
  bucket 381: 90%
  bucket 434: 10%
  bucket 433: 10%
  bucket 380: 100%
  Sorting block of length 582060 for bucket 380
  (Using difference cover)
  bucket 376: 100%
  Sorting block of length 884878 for bucket 376
  (Using difference cover)
  bucket 383: 100%
  Sorting block of length 911992 for bucket 383
  (Using difference

  bucket 417: 50%
  bucket 443: 30%
  bucket 411: 60%
  bucket 434: 40%
  bucket 440: 30%
  bucket 444: 20%
  bucket 407: 60%
  bucket 400: 90%
  bucket 416: 50%
  bucket 394: 100%
  Sorting block of length 479617 for bucket 394
  (Using difference cover)
  bucket 435: 40%
  bucket 419: 60%
  bucket 405: 60%
  bucket 441: 30%
  bucket 445: 20%
  bucket 404: 70%
  bucket 398: 80%
  bucket 450: 10%
  bucket 402: 70%
  bucket 452: 10%
  bucket 451: 10%
  bucket 446: 20%
  bucket 413: 60%
  Sorting block time: 00:00:00
Returning block of 573084 for bucket 392
  bucket 409: 60%
  bucket 408: 60%
  bucket 437: 40%
  bucket 418: 50%
  bucket 436: 50%
  bucket 442: 30%
  bucket 429: 50%
  bucket 448: 20%
  bucket 453: 10%
  bucket 455: 10%
  bucket 454: 10%
  bucket 395: 90%
Getting block 456 of 463
  Reserving size (921105) for bucket 456
  Calculating Z arrays for bucket 456
  Entering block accumulator loop for bucket 456:
  bucket 428: 50%
  bucket 403: 70%
  Sorting block time: 00:00:01
R

  bucket 457: 40%
  bucket 440: 70%
  bucket 448: 60%
  bucket 416: 90%
  bucket 459: 30%
  bucket 452: 50%
  bucket 401: 100%
  Sorting block of length 759287 for bucket 401
  (Using difference cover)
  bucket 451: 50%
  bucket 436: 100%
  Sorting block of length 448125 for bucket 436
  (Using difference cover)
  Sorting block time: 00:00:00
Returning block of 814191 for bucket 419
  bucket 445: 60%
  bucket 458: 40%
  bucket 408: 100%
  Sorting block of length 882801 for bucket 408
  (Using difference cover)
  bucket 455: 60%
  bucket 429: 90%
  bucket 463: 20%
  bucket 447: 70%
  bucket 437: 80%
  bucket 454: 50%
  bucket 409: 100%
  Sorting block of length 719054 for bucket 409
  (Using difference cover)
  bucket 453: 50%
  bucket 462: 20%
  Sorting block time: 00:00:01
Returning block of 828411 for bucket 403
  bucket 420: 90%
  bucket 456: 50%
  bucket 442: 70%
  bucket 428: 90%
  bucket 438: 80%
  bucket 423: 90%
  bucket 414: 100%
  Sorting block of length 715337 for bucket 414

  Sorting block time: 00:00:00
Returning block of 837832 for bucket 449
  bucket 457: 90%
  bucket 453: 80%
  Sorting block time: 00:00:00
Returning block of 494410 for bucket 455
  bucket 450: 100%
  Sorting block of length 876713 for bucket 450
  (Using difference cover)
  bucket 463: 80%
  bucket 460: 80%
  bucket 461: 70%
  bucket 454: 80%
  Sorting block time: 00:00:01
Returning block of 790915 for bucket 446
  Sorting block time: 00:00:01
Returning block of 552637 for bucket 448
  Sorting block time: 00:00:01
Returning block of 627561 for bucket 439
  bucket 445: 100%
  Sorting block of length 864691 for bucket 445
  (Using difference cover)
  bucket 462: 60%
  bucket 459: 80%
  Sorting block time: 00:00:01
Returning block of 665994 for bucket 440
  bucket 458: 90%
  bucket 452: 90%
  Sorting block time: 00:00:01
Returning block of 843692 for bucket 438
  bucket 456: 100%
  Sorting block of length 203601 for bucket 456
  (Using difference cover)
  bucket 451: 90%
  Sorting block 

In [35]:
#cmd

In [36]:
#!bowtie2-build --help

In [37]:
asdasd

NameError: name 'asdasd' is not defined

### scratch

In [ ]:
!gunzip --keep /cellar/users/btsui/Data/ensembl/snp_masked/Mus_musclus.microbe.fa

In [ ]:
!ls -lah /cellar/users/btsui/Data/BOWTIE_GENOME_SNP_INDEX/Homo_sapiens/

In [ ]:
!ls /cellar/users/btsui/Data/ensembl/snp_masked/Mus_musclus.microbe.fa